# Data Analysis

In [15]:
import os
import glob
import shutil
import numpy as np
from edd.data import IBMQData
import edd.workflow as wf

cwd = os.getcwd()
device = "Jakarta"

base_dir = f"{cwd}/{device}/HaarDelayData"

## Step 1: Load in Raw Data and Parse It

In [16]:
result_dir = f"{base_dir}/rawHaarDelayData"
files = glob.glob(f"{result_dir}/*.yml")
len(files)

404

In [17]:
file_dict = {}
for f in files:
    split_f = f.split("_")
    # extract experiment duration label
    tLab_idx = split_f.index("tLab") + 1
    tLab = split_f[tLab_idx]
    # extract whether symmetric or not
    if "asym" in f:
        sym_type = "asym"
    else:
        sym_type = "sym"
        
    key = (tLab, sym_type)
    if key in file_dict:
        file_dict[key].append(f)
    else:
        file_dict[key] = [f]

In [18]:
def compute_fid(counts):
    """
    Computes fidelity as num0 / (num0 + num1).
    """
    num0 = counts.get('0x0', 0)
    num1 = counts.get('0x1', 0)
    
    return num0 / (num0 + num1)


#def compute_error(counts, resamples=1000):
    """
    Computes 2 \sigma error on fidelity.
    """
#    num0 = counts.get('0x0', 0)
#    num1 = counts.get('0x1', 0)
#    tot_counts = num0 + num1
#    prob0 = num0 / tot_counts
#    
#    fids = []
#    for j in range(resamples):
#        re_counts = np.random.choice([0, 1], size=tot_counts, p=[prob0, 1 - prob0]) 
#        fid = 1 - sum(re_counts) / tot_counts
#        fids.append(fid)
#        
#    return 2 * np.std(fids)

def compute_error(counts, resamples=1000):
    """
    Faster way compute errors.
    """
    return 0.0033

## Extract data

In [19]:
data_dict = {}
for key in file_dict:
    # extract key info and files
    t_label = key[0]
    sym_type = key[1]
    file_list = file_dict[key]
    # iterate over files
    for fname in file_list:
        # load and format data
        data = IBMQData(None)
        data.load_raw_data(fname)
        data.format_data()
        for tup in data.tuple_data:
            name, _, counts = tup
            # extract info from name
            split = name.split("_")
            seq_idx = split.index("seq") + 1
            seq = split[seq_idx]
            if seq == "uddx":
                seq += split[seq_idx + 1]
            elif seq == "qdd":
                seq += f"-{split[seq_idx + 1]}-{split[seq_idx + 2]}"
            elif seq == "super":
                seq += f"-{s_info[split + 1]}"
            delay_idx = split.index("delay") + 1
            delay = float(split[delay_idx])
            state_idx = split.index("state") + 1
            state = int(split[state_idx])
            fid = compute_fid(counts)
            error = compute_error(counts, 250)
            count = 0
            key = (seq, t_label, sym_type, state, delay, count)
            while key in data_dict:
                count += 1
                key = (seq, t_label, sym_type, state, delay, count)
            data_dict[key] = (fid, error)

## Save data as CSV Files

In [20]:
# Armonk
#list_of_short_seq = ["ur20", "qdd-1-3", "purex", "xy4", "free"]
#list_of_long_seq = ["ur20", "qdd-1-7", "purex", "xy4", "free"]
# Bogota
#list_of_short_seq = ["ur20", "qdd-1-2", "purex", "xy4", "free"]
#list_of_long_seq = ["ur20", "qdd-3-5", "purex", "xy4", "free"]
# Jakarta
list_of_short_seq = ['ur10', 'qdd-1-6', 'purex', 'xy4', 'free']
list_of_long_seq = ['ur10', 'qdd-1-6', 'purex', 'xy4', 'free']

In [21]:
# first save the short time files
t_lab = "short"
for seq in list_of_short_seq:
    for sym_type in ["sym", "asym"]:
        for state in range(25):
            file_prefix = f"{base_dir}/csvHaarDelayData/haarDelayData_{device}_seq_{seq}_state_{state}"
            is_sym = sym_type == "sym"
            file_prefix += f"_sym_{is_sym}_tLab_{t_lab}"
            d_key = (seq, t_lab, sym_type, state)
            delays = {}
            fids = {}
            errs = {}
            counts = []
            for key, fid_tup in data_dict.items():
                keys_match = all([key[j] == d_key[j] for j in range(len(d_key))])
                if keys_match:
                    count = key[-1]
                    if count not in counts:
                        counts.append(count)
                        delays[count] = [key[4]]
                        fids[count] = [fid_tup[0]]
                        errs[count] = [fid_tup[1]]
                    else:
                        delays[count].append(key[4])
                        fids[count].append(fid_tup[0])
                        errs[count].append(fid_tup[1])

            for c in counts:
                fname = f"{file_prefix}_count_{c}.csv"
                with open(fname, 'w') as f:
                    for j in range(len(delays[c])):
                        f.write(f"{delays[c][j]},{fids[c][j]},{errs[c][j]}\n")
                        
                        
# next save the long time files
t_lab = "long"
for seq in list_of_long_seq:
    for sym_type in ["sym", "asym"]:
        for state in range(25):
            file_prefix = f"{base_dir}/csvHaarDelayData/haarDelayData_{device}_seq_{seq}_state_{state}"
            is_sym = sym_type == "sym"
            file_prefix += f"_sym_{is_sym}_tLab_{t_lab}"
            d_key = (seq, t_lab, sym_type, state)
            delays = {}
            fids = {}
            errs = {}
            counts = []
            for key, fid_tup in data_dict.items():
                keys_match = all([key[j] == d_key[j] for j in range(len(d_key))])
                if keys_match:
                    count = key[-1]
                    if count not in counts:
                        counts.append(count)
                        delays[count] = [key[4]]
                        fids[count] = [fid_tup[0]]
                        errs[count] = [fid_tup[1]]
                    else:
                        delays[count].append(key[4])
                        fids[count].append(fid_tup[0])
                        errs[count].append(fid_tup[1])

            for c in counts:
                fname = f"{file_prefix}_count_{c}.csv"
                with open(fname, 'w') as f:
                    for j in range(len(delays[c])):
                        f.write(f"{delays[c][j]},{fids[c][j]},{errs[c][j]}\n")

### Step 7: Go and retrieve old Haar CSV results for Free and wrangle new one into old format

In [199]:
old_short_file1 = f"{cwd}/{device}/HaarData/csvHaarData/optDelay_haarData_Jakarta.csv"
with open(old_short_file1, 'r') as f:
    short_free_data1 = f.readlines()[-1]
    
#old_short_file2 = f"{cwd}/{device}/HaarData/alsoOldCsvHaarData/haarData_Bogota.csv"
#with open(old_short_file2, 'r') as f:
#    short_free_data2 = f.readlines()[-1]
    
#old_short_file3 = f"{cwd}/{device}/HaarData/oldCsvHaarData/haarData_Bogota.csv"
#with open(old_short_file3, 'r') as f:
#    short_free_data3 = f.readlines()[7]
    
#old_short_file4 = f"{cwd}/{device}/HaarData/oldCsvHaarData/bogota_2021-11-06_19_17_55_csvFidList.csv"
#with open(old_short_file4, 'r') as f:
#    short_free_data4 = f.readlines()[-1]

#full_data = short_free_data1[5:-1] + "," + short_free_data3[5:-1] + "," + short_free_data4[5:-1]
full_data = short_free_data1[5:-1]
print(len(full_data.split(',')) / 30)

new_short_file = f"{cwd}/{device}/HaarDelayData/csvHaarFreeData/free_short_Jakarta.csv"
with open(new_short_file, 'w') as f:
    f.write(full_data)

3.3333333333333335


In [222]:
long_free_fids = []
for f in files:
    if "free" in f:
        data = IBMQData(None)
        data.load_raw_data(f)
        data.format_data()
        for tup in data.tuple_data:
            _, _, counts = tup
            fid = compute_fid(counts)
            long_free_fids.append(fid)
            
long_file = f"{cwd}/{device}/HaarDelayData/csvHaarFreeData/free_long_Jakarta.csv"
with open(long_file, 'w') as f:
    f.write(','.join([str(x) for x in long_free_fids]))

### Step 8: Extract old CSV data from Jakarta for current plot proxy

In [ ]:
model = "haarDelayData_Jakarta_seq_free_state_0_sym_False_tLab_long_count_0"

In [214]:
seqs_of_interest = ["purex", "xy4", "ur10", "qdd-1-6"]
old_csv_files = glob.glob(f"{cwd}/{device}/DelayData/csvDelayData/*.csv")
for f in old_csv_files:
    relevant = any([seqs_of_interest[i] in f for i in range(4)])
    if relevant:
        split_f = f.split("_")
        sym_idx = split_f.index("sym")
        p1 = split_f[0:sym_idx + 2]
        p2 = split_f[sym_idx + 2: ]
        new_split = p1 + ["tLab", "short"] + p2
        new_f = "_".join(new_split)
        new_f = new_f.replace("DelayData", "HaarDelayData")
        shutil.copy(f, new_f)

In [ ]:
shutil.copy(f, new_f)

In [ ]:
old_short_file1 = f"{cwd}/{device}/
with open(old_short_file1, 'r') as f:
    short_free_data1 = f.readlines()[-1]

#full_data = short_free_data1[5:-1] + "," + short_free_data3[5:-1] + "," + short_free_data4[5:-1]
full_data = short_free_data1[5:-1]
print(len(full_data.split(',')) / 30)

new_short_file = f"{cwd}/{device}/HaarDelayData/csvHaarFreeData/free_short_Jakarta.csv"
with open(new_short_file, 'w') as f:
    f.write(full_data)

### Step 6: (not necessary anymore) Turn _ into - for sequence names in csv handles

In [228]:
files = glob.glob(f"{base_dir}/csvHaarDelayData/*.csv")
import os

In [231]:
for f in files:
    split_f = f.split("_")
    seq_idx = split_f.index("seq") + 1
    seq = split_f[seq_idx]
    if seq == "qdd":
        full_seq = f"{seq}-{split_f[seq_idx + 1]}-{split_f[seq_idx + 2]}"
        split_f[seq_idx] = full_seq
        split_f[seq_idx + 1] = None
        split_f[seq_idx + 2] = None
        split_f.remove(None)
        split_f.remove(None)
        
    if seq == "super":
        full_seq = f"{seq}-{split_f[seq_idx + 1]}"
        split_f[seq_idx] = full_seq
        split_f[seq_idx + 1] = None
        split_f.remove(None)
    new_f = "_".join(split_f)
    
    os.rename(f, new_f)

In [232]:
files = glob.glob(f"{base_dir}/rawHaarDelayData/*.yml")
import os

In [241]:
for f in files:
    split_f = f.split("_")
    seq_idx = split_f.index("Paper") + 4
    seq = split_f[seq_idx]
    if seq == "qdd":
        full_seq = f"{seq}-{split_f[seq_idx + 1]}-{split_f[seq_idx + 2]}"
        split_f[seq_idx] = full_seq
        split_f[seq_idx + 1] = None
        split_f[seq_idx + 2] = None
        split_f.remove(None)
        split_f.remove(None)
        
    if seq == "super":
        full_seq = f"{seq}-{split_f[seq_idx + 1]}"
        split_f[seq_idx] = full_seq
        split_f[seq_idx + 1] = None
        split_f.remove(None)
    new_f = "_".join(split_f)
    
    os.rename(f, new_f)

In [69]:
data_dir = "/Users/worknic/Documents/Armonk_Bogota_24AugData/Bogota/q3/min_delay/"

for file in data_files:
    file = file + ".csv"
    shutil.copy(file, data_dir)

NameError: name 'data_files' is not defined

In [103]:
[x for x in range(10)][0:4]

[0, 1, 2, 3]

In [104]:
[x for x in range(10)][4:8]

[4, 5, 6, 7]

## Step ?: Fix file names if necessary

In [226]:
result_dir = f"{base_dir}/rawHaarDelayData"
files = glob.glob(f"{result_dir}/*.yml")
for f in files[0:5]:
    sp_f = f.split('_')
    job_idx = sp_f.index('job')
    print(sp_f)
    for j in range(2):
        sp_f[job_idx + 2 + j] = None
    print(sp_f)
    for j in range(2):
        sp_f.remove(None)
    new_f = "_".join(sp_f)
    print(f)
    print(new_f)
    #os.rename(f, new_f)

['/Users/worknic/Dropbox (Lidar group)/EDD', 'Paper', 'Data/Jakarta/HaarDelayData/rawHaarDelayData/jakarta', '2022-01-13', '17:32:28', 'free', 'job', '1', 'sym', 'tLab', 'long', 'rawData.yml']
['/Users/worknic/Dropbox (Lidar group)/EDD', 'Paper', 'Data/Jakarta/HaarDelayData/rawHaarDelayData/jakarta', '2022-01-13', '17:32:28', 'free', 'job', '1', None, None, 'long', 'rawData.yml']
/Users/worknic/Dropbox (Lidar group)/EDD_Paper_Data/Jakarta/HaarDelayData/rawHaarDelayData/jakarta_2022-01-13_17:32:28_free_job_1_sym_tLab_long_rawData.yml
/Users/worknic/Dropbox (Lidar group)/EDD_Paper_Data/Jakarta/HaarDelayData/rawHaarDelayData/jakarta_2022-01-13_17:32:28_free_job_1_long_rawData.yml
['/Users/worknic/Dropbox (Lidar group)/EDD', 'Paper', 'Data/Jakarta/HaarDelayData/rawHaarDelayData/jakarta', '2022-01-13', '17:32:28', 'qdd', '1', '6', 'job', '2', 'asym', 'tLab', 'long', 'rawData.yml']
['/Users/worknic/Dropbox (Lidar group)/EDD', 'Paper', 'Data/Jakarta/HaarDelayData/rawHaarDelayData/jakarta', '2

In [ ]:
csv_files = glob.glob(f"{base_dir}/csvHaarDelayData/*.csv")
for f in csv_files:
    os.remove(f)